In [5]:
import torch
from torch import nn, optim
import os
from torch.utils.data import DataLoader
from tqdm import tqdm
import pandas as pd

In [8]:
ids = pd.read_csv("data/IdLookupTable.csv", index_col = False)
ids.head(10)

,RowId,ImageId,FeatureName,Location
0,1,1,left_eye_center_x,NaN
1,2,1,left_eye_center_y,NaN
2,3,1,right_eye_center_x,NaN
3,4,1,right_eye_center_y,NaN
4,5,1,left_eye_inner_corner_x,NaN
5,6,1,left_eye_inner_corner_y,NaN
6,7,1,left_eye_outer_corner_x,NaN
7,8,1,left_eye_outer_corner_y,NaN
8,9,1,right_eye_inner_corner_x,NaN
9,10,1,right_eye_inner_corner_y,NaN


In [12]:
train_df = pd.read_csv("data/training.csv", index_col = False)
train_df.head(3)

,left_eye_center_x,left_eye_center_y,right_eye_center_x,right_eye_center_y,left_eye_inner_corner_x,left_eye_inner_corner_y,left_eye_outer_corner_x,left_eye_outer_corner_y,right_eye_inner_corner_x,right_eye_inner_corner_y,...,nose_tip_y,mouth_left_corner_x,mouth_left_corner_y,mouth_right_corner_x,mouth_right_corner_y,mouth_center_top_lip_x,mouth_center_top_lip_y,mouth_center_bottom_lip_x,mouth_center_bottom_lip_y,Image
0,66.033564,39.002274,30.227008,36.421678,59.582075,39.647423,73.130346,39.969997,36.356571,37.389402,...,57.066803,61.195308,79.970165,28.614496,77.388992,43.312602,72.935459,43.130707,84.485774,238 236 237 238 240 240 239 241 241 243 240 23...
1,64.332936,34.970077,29.949277,33.448715,58.856170,35.274349,70.722723,36.187166,36.034723,34.361532,...,55.660936,56.421447,76.352000,35.122383,76.047660,46.684596,70.266553,45.467915,85.480170,219 215 204 196 204 211 212 200 180 168 178 19...
2,65.057053,34.909642,30.903789,34.909642,59.412000,36.320968,70.984421,36.320968,37.678105,36.320968,...,53.538947,60.822947,73.014316,33.726316,72.732000,47.274947,70.191789,47.274947,78.659368,144 142 159 180 188 188 184 180 167 132 84 59 ...


In [13]:
train_df.dtypes

left_eye_center_x            float64
left_eye_center_y            float64
right_eye_center_x           float64
right_eye_center_y           float64
left_eye_inner_corner_x      float64
left_eye_inner_corner_y      float64
left_eye_outer_corner_x      float64
left_eye_outer_corner_y      float64
right_eye_inner_corner_x     float64
right_eye_inner_corner_y     float64
right_eye_outer_corner_x     float64
right_eye_outer_corner_y     float64
left_eyebrow_inner_end_x     float64
left_eyebrow_inner_end_y     float64
left_eyebrow_outer_end_x     float64
left_eyebrow_outer_end_y     float64
right_eyebrow_inner_end_x    float64
right_eyebrow_inner_end_y    float64
right_eyebrow_outer_end_x    float64
right_eyebrow_outer_end_y    float64
nose_tip_x                   float64
nose_tip_y                   float64
mouth_left_corner_x          float64
mouth_left_corner_y          float64
mouth_right_corner_x         float64
mouth_right_corner_y         float64
mouth_center_top_lip_x       float64
m

In [17]:
train_df.isna().sum()

left_eye_center_x              10
left_eye_center_y              10
right_eye_center_x             13
right_eye_center_y             13
left_eye_inner_corner_x      4778
left_eye_inner_corner_y      4778
left_eye_outer_corner_x      4782
left_eye_outer_corner_y      4782
right_eye_inner_corner_x     4781
right_eye_inner_corner_y     4781
right_eye_outer_corner_x     4781
right_eye_outer_corner_y     4781
left_eyebrow_inner_end_x     4779
left_eyebrow_inner_end_y     4779
left_eyebrow_outer_end_x     4824
left_eyebrow_outer_end_y     4824
right_eyebrow_inner_end_x    4779
right_eyebrow_inner_end_y    4779
right_eyebrow_outer_end_x    4813
right_eyebrow_outer_end_y    4813
nose_tip_x                      0
nose_tip_y                      0
mouth_left_corner_x          4780
mouth_left_corner_y          4780
mouth_right_corner_x         4779
mouth_right_corner_y         4779
mouth_center_top_lip_x       4774
mouth_center_top_lip_y       4774
mouth_center_bottom_lip_x      33
mouth_center_b

In [27]:
print(len(train_df['Image'][0]))
print(train_df['Image'][0])

33593
238 236 237 238 240 240 239 241 241 243 240 239 231 212 190 173 148 122 104 92 79 73 74 73 73 74 81 74 60 64 75 86 93 102 100 105 109 114 121 127 132 134 137 137 140 139 138 137 137 140 141 143 144 147 148 149 147 147 148 145 147 144 146 147 147 143 134 130 130 128 116 104 98 90 82 78 85 88 86 80 77 87 108 111 115 128 133 188 242 252 250 248 251 250 250 250 235 238 236 238 238 237 238 242 241 239 237 233 215 195 187 156 119 103 93 78 68 73 75 75 72 75 70 61 66 77 91 96 106 108 113 120 125 131 134 138 135 138 139 145 144 144 142 140 141 141 148 147 150 149 152 151 149 150 147 148 144 148 144 146 146 143 139 128 132 135 128 112 104 97 87 78 79 83 85 83 75 75 89 109 111 117 117 130 194 243 251 249 250 249 250 251 237 236 237 238 237 238 241 238 238 238 241 221 195 187 163 124 106 95 81 68 70 73 73 72 73 69 65 74 82 94 103 110 111 119 127 135 140 139 144 144 145 143 143 143 146 144 144 147 146 148 147 150 146 147 150 150 149 147 147 144 147 144 149 146 144 144 144 140 131 133 133 125

In [29]:
train_df.shape

(7049, 31)

In [31]:
train_15_landmarks = train_df[~(train_df.isnull().sum(axis=1) > 20)].copy()
train_4_landmarks = train_df[(train_df.isnull().sum(axis=1) > 20)].copy()

In [34]:
train_15_landmarks.shape, train_4_landmarks.shape

((2284, 31), (4765, 31))

In [37]:
train_15_landmarks.iloc[0:100, :].to_csv("data/val_15.csv")
train_15_landmarks.iloc[100:, :].to_csv("data/train_15.csv")

In [38]:
train_4_landmarks.iloc[0:200, :].to_csv("data/val_4.csv")
train_4_landmarks.iloc[200:, :].to_csv("data/train_4.csv")

In [77]:
train_4 = pd.read_csv("data/train_15.csv", index_col = False)
train_4.head(1)

,Unnamed: 0,left_eye_center_x,left_eye_center_y,right_eye_center_x,right_eye_center_y,left_eye_inner_corner_x,left_eye_inner_corner_y,left_eye_outer_corner_x,left_eye_outer_corner_y,right_eye_inner_corner_x,...,nose_tip_y,mouth_left_corner_x,mouth_left_corner_y,mouth_right_corner_x,mouth_right_corner_y,mouth_center_top_lip_x,mouth_center_top_lip_y,mouth_center_bottom_lip_x,mouth_center_bottom_lip_y,Image
0,100,64.540235,33.425647,31.731529,35.949318,59.492471,34.372024,69.902824,33.110188,35.832706,...,58.662776,68.325882,69.703765,34.886118,73.173882,50.659765,67.180235,50.659765,79.168235,254 251 251 252 236 181 106 67 57 55 73 83 74 ...


In [59]:
import numpy as np
np.array(train_4.iloc[1,30].split(), dtype=np.float32)

AttributeError: 'numpy.float64' object has no attribute 'split'

In [62]:
train_4.iloc[2,31]

'128 141 132 145 140 148 184 214 216 221 227 227 224 224 226 228 230 232 234 236 238 240 241 242 243 243 244 244 245 246 247 247 248 247 247 246 246 247 247 247 246 245 244 244 244 244 244 245 245 245 242 241 242 243 243 243 242 241 242 241 239 237 238 239 240 240 239 238 237 238 237 235 234 231 230 229 228 225 220 213 205 197 190 182 178 181 177 171 168 169 169 166 159 154 157 166 136 138 132 133 125 141 186 217 215 218 226 228 224 222 224 228 229 230 232 235 237 239 240 241 245 245 244 244 245 246 247 248 249 248 248 248 248 248 248 248 248 247 246 246 246 246 246 246 245 245 243 244 245 244 243 242 243 242 240 244 242 240 239 239 239 240 240 239 238 238 237 237 236 233 231 230 227 225 221 216 209 201 191 186 182 185 177 170 168 170 171 168 162 156 152 160 112 144 147 134 121 145 189 214 214 217 224 226 225 222 222 228 229 230 231 233 235 239 241 241 244 244 244 244 245 246 247 249 250 249 249 249 249 250 249 249 250 249 248 248 248 248 248 247 247 247 245 245 246 244 242 242 243 242

In [78]:
train_4.head(1)

,Unnamed: 0,left_eye_center_x,left_eye_center_y,right_eye_center_x,right_eye_center_y,left_eye_inner_corner_x,left_eye_inner_corner_y,left_eye_outer_corner_x,left_eye_outer_corner_y,right_eye_inner_corner_x,...,nose_tip_y,mouth_left_corner_x,mouth_left_corner_y,mouth_right_corner_x,mouth_right_corner_y,mouth_center_top_lip_x,mouth_center_top_lip_y,mouth_center_bottom_lip_x,mouth_center_bottom_lip_y,Image
0,100,64.540235,33.425647,31.731529,35.949318,59.492471,34.372024,69.902824,33.110188,35.832706,...,58.662776,68.325882,69.703765,34.886118,73.173882,50.659765,67.180235,50.659765,79.168235,254 251 251 252 236 181 106 67 57 55 73 83 74 ...


In [79]:
train_4.drop(columns="Unnamed: 0")

,left_eye_center_x,left_eye_center_y,right_eye_center_x,right_eye_center_y,left_eye_inner_corner_x,left_eye_inner_corner_y,left_eye_outer_corner_x,left_eye_outer_corner_y,right_eye_inner_corner_x,right_eye_inner_corner_y,...,nose_tip_y,mouth_left_corner_x,mouth_left_corner_y,mouth_right_corner_x,mouth_right_corner_y,mouth_center_top_lip_x,mouth_center_top_lip_y,mouth_center_bottom_lip_x,mouth_center_bottom_lip_y,Image
0,64.540235,33.425647,31.731529,35.949318,59.492471,34.372024,69.902824,33.110188,35.832706,35.949318,...,58.662776,68.325882,69.703765,34.886118,73.173882,50.659765,67.180235,50.659765,79.168235,254 251 251 252 236 181 106 67 57 55 73 83 74 ...
1,64.354588,37.862118,28.815529,38.179765,58.642588,38.497412,70.701176,38.497412,35.796000,38.179765,...,56.266588,63.402353,80.383059,39.921176,82.286824,53.565882,74.671059,55.152706,89.267294,26 29 27 24 28 23 21 24 25 25 23 24 27 27 26 2...
2,60.278634,33.919376,27.840000,40.895454,55.744000,34.616976,66.208780,32.872898,33.769366,39.849054,...,51.323317,69.347902,69.846634,42.489756,75.427122,55.744000,69.148722,57.837268,77.869268,39 41 51 59 61 64 65 65 61 60 64 63 66 78 84 7...
3,64.556883,38.323948,31.404468,40.831169,58.149195,39.437922,70.685922,38.881247,37.533506,41.109818,...,50.582026,64.835532,74.540883,33.911688,76.769455,48.676987,68.411844,49.234286,78.162701,175 176 176 175 175 177 175 175 175 176 172 16...
4,69.123692,38.836923,30.345846,39.387077,63.622769,39.387077,75.723692,39.937231,36.945846,39.387077,...,55.888615,63.347692,75.139692,30.895385,72.389538,45.746462,71.289231,44.371692,81.190154,170 170 168 166 166 163 160 159 154 144 134 12...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2179,62.965072,35.235867,32.983512,35.915684,54.757421,37.663784,70.229488,36.086712,39.223046,38.093504,...,60.426483,62.918915,71.901758,35.620390,71.940207,47.251100,74.208676,47.308773,75.304462,158 158 159 159 160 160 159 162 165 122 63 88 ...
2180,64.884689,30.735699,29.548608,32.871821,56.547124,32.682915,73.013274,30.906074,35.931236,33.952750,...,56.255015,65.931517,70.404343,32.971871,72.756972,48.069810,74.675621,48.298221,74.949713,9 22 26 21 25 15 23 18 16 25 46 17 17 12 16 13...
2181,68.430866,38.651975,28.895857,37.617027,61.659350,40.100902,75.586792,40.219182,37.665118,39.087245,...,63.289576,68.864397,77.495823,22.013981,77.368995,49.180628,79.043130,47.176739,89.544522,31 40 47 31 54 58 63 100 86 80 82 75 79 86 90 ...
2182,64.152180,30.691592,27.000898,40.868082,56.505624,34.126963,73.436776,28.556335,34.746122,40.506939,...,59.735799,73.730743,70.273886,38.777143,80.684286,58.042857,79.301429,58.611086,80.355543,7 1 5 1 3 20 12 0 5 8 6 18 13 9 8 12 12 11 4 8...


Installing EfficientNet

In [68]:
import sys
sys.path.append('efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master')
from efficientnet_pytorch import EfficientNet


In [69]:
EfficientNet

efficientnet_pytorch.model.EfficientNet

In [70]:
sys.path

['/Users/ayushgupta/Ayush/GitHub/Facial Keypoint Detection',
 '/Users/ayushgupta/anaconda3/lib/python310.zip',
 '/Users/ayushgupta/anaconda3/lib/python3.10',
 '/Users/ayushgupta/anaconda3/lib/python3.10/lib-dynload',
 '',
 '/Users/ayushgupta/anaconda3/lib/python3.10/site-packages',
 '/Users/ayushgupta/anaconda3/lib/python3.10/site-packages/PyQt5_sip-12.11.0-py3.10-macosx-11.1-arm64.egg',
 '/Users/ayushgupta/anaconda3/lib/python3.10/site-packages/aeosa',
 '/Users/ayushgupta/Ayush/Data Science/Radiology',
 '/Users/ayushgupta/anaconda3/lib/python3.10/site-packages/mpmath-1.2.1-py3.10.egg',
 '/Users/ayushgupta/anaconda3/lib/python3.10/site-packages/pycurl-7.45.1-py3.10-macosx-11.1-arm64.egg',
 '../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master',
 '../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master',
 'efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master']

In [83]:
#from torchsummary import summary

model = EfficientNet.from_pretrained("efficientnet-b0")
model._fc = nn.Linear(1280, 30)
model

Loaded pretrained weights for efficientnet-b0


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
    